### 结合 DeepSeek RAG 课程中学习到的方法，将示例代码中的模拟工具 mock\_query\_product\_database 替换为真实的产品 / 商品数据

In [1]:
import os
from openai import OpenAI

# 建议将 API Key 设置为环境变量，避免直接暴露在代码中
# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
base_url = os.getenv("DEEPSEEK_BASE_URL")
if not api_key:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")

# 初始化 DeepSeek 客户端

client = OpenAI(
    api_key=api_key,
    base_url=base_url,  # DeepSeek API 的基地址
)

In [2]:
SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新潮流和产品卖点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

请始终采用'Thought-Action-Observation'模式进行推理和行动。文案风格需活泼、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```
在生成文案前，请务必先思考并收集足够的信息。
"""

In [3]:
TOOLS_DEFINITION = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "搜索互联网上的实时信息，用于获取最新新闻、流行趋势、用户评价、行业报告等。请确保搜索关键词精确，避免宽泛的查询。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "要搜索的关键词或问题，例如'最新小红书美妆趋势'或'深海蓝藻保湿面膜 用户评价'"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "query_product_database",
            "description": "查询内部产品数据库，获取指定产品的详细卖点、成分、适用人群、使用方法等信息。",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "要查询的产品名称，例如'深海蓝藻保湿面膜'"
                    }
                },
                "required": ["product_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_emoji",
            "description": "根据提供的文本内容，生成一组适合小红书风格的表情符号。",
            "parameters": {
                "type": "object",
                "properties": {
                    "context": {
                        "type": "string",
                        "description": "文案的关键内容或情感，例如'惊喜效果'、'补水保湿'"
                    }
                },
                "required": ["context"]
            }
        }
    }
]

### 将大模型生成的产品数据导入向量数据库

#### 提示词

你是一个专业的产品数据生成大师，很轻松的能够生成大量的产品信息
产品信息用 \n-------\n 分隔 ，产品信息用markdown格式来写
每次生成完整的10条产品信息，不能重复

以下是一些可供参考的案例：
1. 深海蓝藻保湿面膜：核心成分为深海蓝藻提取物，富含多糖和氨基酸，能深层补水、修护肌肤屏障、舒缓敏感泛红。质地清爽不粘腻，适合所有肤质，尤其适合干燥、敏感肌。规格：25ml*5片。
2. 美白精华：核心成分是烟酰胺和VC衍生物，主要功效是提亮肤色、淡化痘印、改善暗沉。质地轻薄易吸收，适合需要均匀肤色的人群。
3. 玻尿酸补水精华：核心成分为高浓度玻尿酸，具有强大的保湿功效，能够迅速为肌肤补充水分，提升肌肤弹性和光泽。质地轻盈，易于被肌肤吸收，适合各种肤质，特别适合干燥缺水的肌肤。规格：30ml。
4. 抗氧化精华：主要成分包括维生素C和辅酶Q10，能够有效对抗自由基，延缓肌肤衰老，同时改善肌肤暗沉问题。质地丝滑不粘腻，适合想要抗衰老和提亮肤色的人群使用。规格：30ml。


#### 生成的数据
数据放在当前目录下，文件名为：product_data.md

In [4]:
# 准备数据
products = []
product_file_path = "./product_data.md"
with open(product_file_path, "r") as file:
    file_text: str = file.read()
    # 找到所有商品
    products += file_text.split("\n-------\n")

In [5]:
# 验证数据是否准备OK
for i in range(0, 3):
    print(products[i])

**胶原蛋白紧致面膜**  
核心成分为水解胶原蛋白、胜肽复合物，能够提升肌肤弹性，淡化细纹，改善松弛状况。质地柔软贴肤，适合熟龄肌使用。  
规格：28ml*5片。

**绿茶清爽控油洁面乳**  
主要成分为绿茶提取物与水杨酸，有效清洁毛孔、控制油脂分泌，减少黑头和粉刺生成。泡沫细腻，适合油性及混合性肌肤使用。  
规格：120ml。

**玫瑰纯露柔肤水**  
选用保加利亚玫瑰蒸馏而成，具有补水、舒缓、提亮肤色等多重功效，温和无刺激。可作为日常喷雾或二次清洁水使用。  
规格：200ml。



In [6]:
# 准备embedding模型
from pymilvus import model as milvus_model

embedding_model = milvus_model.DefaultEmbeddingFunction()

/root/miniconda3/envs/deepseek-quickstart/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

768
[-0.04836059  0.07163021 -0.01130063 -0.03789341 -0.03320651 -0.01318453
 -0.03041721 -0.02269495 -0.02317858 -0.00426026]


In [8]:
# 将商品数据加载到Milvus
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_product.db")
collection_name = "production_collection"

In [22]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

In [23]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",
    consistency_level="Strong"
)

In [24]:
# 插入数据
from tqdm import tqdm

data = []
doc_embeddings = embedding_model.encode_documents(products)

for i, product in enumerate(tqdm(products, desc="Create embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": product})

milvus_client.insert(collection_name=collection_name, data=data)

Create embeddings: 100%|██████████| 50/50 [00:00<00:00, 102851.99it/s]


{'insert_count': 50, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49], 'cost': 0}

In [25]:
# 从向量数据库搜索产品信息

def query_product_database_by_milvus(product_name: str) -> str:
    print(f"[Tool Call] 查询Milvus产品数据库：{product_name}")
    search_res = milvus_client.search(
        collection_name=collection_name,
        data=embedding_model.encode_queries([product_name]),
        limit=1,
        search_param={"metric_type": "IP", "params": {}},
        output_fields=["text"]
    )
    if len(search_res) == 0:
        return f"产品数据库中未找到关于 '{product_name}' 的详细信息。"
    res = search_res[0]
    if len(res) == 0:
        return f"产品数据库中未找到关于 '{product_name}' 的详细信息。"
    return res[0]["entity"]["text"]

In [26]:
import random  # 用于模拟生成表情
import time  # 用于模拟网络延迟


def mock_search_web(query: str) -> str:
    """模拟网页搜索工具，返回预设的搜索结果。"""
    print(f"[Tool Call] 模拟搜索网页：{query}")
    time.sleep(1)  # 模拟网络延迟
    if "小红书美妆趋势" in query:
        return "近期小红书美妆流行'多巴胺穿搭'、'早C晚A'护肤理念、'伪素颜'妆容，热门关键词有#氛围感、#抗老、#屏障修复。"
    elif "保湿面膜" in query:
        return "小红书保湿面膜热门话题：沙漠干皮救星、熬夜急救面膜、水光肌养成。用户痛点：卡粉、泛红、紧绷感。"
    elif "深海蓝藻保湿面膜" in query:
        return "关于深海蓝藻保湿面膜的用户评价：普遍反馈补水效果好，吸收快，对敏感肌友好。有用户提到价格略高，但效果值得。"
    elif "抗氧化精华" in query:
        return "小红书上抗氧化精华的热门话题集中在成分解析、长效抗氧化以及肤质匹配。用户关心的问题多数是效果持续性、质地轻盈与否以及是否适合混合肌。"
    elif "玻尿酸补水精华" in query:
        return "关于玻尿酸补水精华的反馈，用户普遍认为它具有极佳的锁水能力，能够迅速改善肌肤干燥问题。不过，一些用户提到需要持续使用才能看到效果。"
    elif "茶树控油面膜" in query:
        return "小红书用户对茶树控油面膜的评价集中于其控油去痘的效果，且能有效清洁毛孔细致肌肤。对油性和混合性肌肤友好。用户提到关键词如#清爽控油、#痘肌救星、#毛孔清洁。"
    else:
        return f"未找到关于 '{query}' 的特定信息，但市场反馈通常关注产品成分、功效和用户体验。"


def mock_query_product_database(product_name: str) -> str:
    """模拟查询产品数据库，返回预设的产品信息。"""
    print(f"[Tool Call] 模拟查询产品数据库：{product_name}")
    time.sleep(0.5)  # 模拟数据库查询延迟
    if "深海蓝藻保湿面膜" in product_name:
        return "深海蓝藻保湿面膜：核心成分为深海蓝藻提取物，富含多糖和氨基酸，能深层补水、修护肌肤屏障、舒缓敏感泛红。质地清爽不粘腻，适合所有肤质，尤其适合干燥、敏感肌。规格：25ml*5片。"
    elif "美白精华" in product_name:
        return "美白精华：核心成分是烟酰胺和VC衍生物，主要功效是提亮肤色、淡化痘印、改善暗沉。质地轻薄易吸收，适合需要均匀肤色的人群。"
    if "玻尿酸补水精华" in product_name:
        return "玻尿酸补水精华：核心成分为高浓度玻尿酸，具有强大的保湿功效，能够迅速为肌肤补充水分，提升肌肤弹性和光泽。质地轻盈，易于被肌肤吸收，适合各种肤质，特别适合干燥缺水的肌肤。规格：30ml。"
    elif "抗氧化精华" in product_name:
        return "抗氧化精华：主要成分包括维生素C和辅酶Q10，能够有效对抗自由基，延缓肌肤衰老，同时改善肌肤暗沉问题。质地丝滑不粘腻，适合想要抗衰老和提亮肤色的人群使用。规格：30ml。"
    else:
        return f"产品数据库中未找到关于 '{product_name}' 的详细信息。"


def mock_generate_emoji(context: str) -> list:
    """模拟生成表情符号，根据上下文提供常用表情。"""
    print(f"[Tool Call] 模拟生成表情符号，上下文：{context}")
    time.sleep(0.2)  # 模拟生成延迟
    if "补水" in context or "水润" in context or "保湿" in context:
        return ["💦", "💧", "🌊", "✨"]
    elif "惊喜" in context or "哇塞" in context or "爱了" in context:
        return ["💖", "😍", "🤩", "💯"]
    elif "熬夜" in context or "疲惫" in context:
        return ["😭", "😮‍💨", "😴", "💡"]
    elif "好物" in context or "推荐" in context:
        return ["✅", "👍", "⭐", "🛍️"]
    elif "控油" in context or "清爽" in context:
        return ["🌿", "🍃", "✨", "💧"]
    elif "效果显著" in context or "明显改善" in context:
        return ["👏", "💪", "👍", "🎉"]
    elif "舒缓" in context or "放松" in context:
        return ["😌", "🌸", "🧘", "🛁"]
    elif "自然" in context or "无添加" in context:
        return ["🍀", "🌼", "🌿", "💚"]
    else:
        return random.sample(["✨", "🔥", "💖", "💯", "🎉", "👍", "🤩", "💧", "🌿"], k=min(5, len(context.split())))


# 将模拟工具函数映射到一个字典，方便通过名称调用
available_tools = {
    "search_web": mock_search_web,
    "query_product_database": query_product_database_by_milvus,
    "generate_emoji": mock_generate_emoji,
}

In [27]:
import json
import re


def generate_rednote(product_name: str, tone_style: str = "活泼甜美", max_iterations: int = 5) -> str:
    """
    使用 DeepSeek Agent 生成小红书爆款文案。
    
    Args:
        product_name (str): 要生成文案的产品名称。
        tone_style (str): 文案的语气和风格，如"活泼甜美"、"知性"、"搞怪"等。
        max_iterations (int): Agent 最大迭代次数，防止无限循环。
        
    Returns:
        str: 生成的爆款文案（JSON 格式字符串）。
    """

    print(f"\n🚀 启动小红书文案生成助手，产品：{product_name}，风格：{tone_style}\n")

    # 存储对话历史，包括系统提示词和用户请求
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",
         "content": f"请为产品「{product_name}」生成一篇小红书爆款文案。要求：语气{tone_style}，包含标题、正文、至少5个相关标签和5个表情符号。请以完整的JSON格式输出，并确保JSON内容用markdown代码块包裹（例如：```json{{...}}```）。"}
    ]

    iteration_count = 0
    final_response = None

    while iteration_count < max_iterations:
        iteration_count += 1
        print(f"-- Iteration {iteration_count} --")

        try:
            # 调用 DeepSeek API，传入对话历史和工具定义
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=messages,
                tools=TOOLS_DEFINITION,  # 告知模型可用的工具
                tool_choice="auto"  # 允许模型自动决定是否使用工具
            )

            response_message = response.choices[0].message

            # **ReAct模式：处理工具调用**
            if response_message.tool_calls:  # 如果模型决定调用工具
                print("Agent: 决定调用工具...")
                messages.append(response_message)  # 将工具调用信息添加到对话历史

                tool_outputs = []
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    # 确保参数是合法的JSON字符串，即使工具不要求参数，也需要传递空字典
                    function_args = json.loads(tool_call.function.arguments) if tool_call.function.arguments else {}

                    print(f"Agent Action: 调用工具 '{function_name}'，参数：{function_args}")

                    # 查找并执行对应的模拟工具函数
                    if function_name in available_tools:
                        tool_function = available_tools[function_name]
                        tool_result = tool_function(**function_args)
                        print(f"Observation: 工具返回结果：{tool_result}")
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": str(tool_result)  # 工具结果作为字符串返回
                        })
                    else:
                        error_message = f"错误：未知的工具 '{function_name}'"
                        print(error_message)
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": error_message
                        })
                messages.extend(tool_outputs)  # 将工具执行结果作为 Observation 添加到对话历史

            # **ReAct 模式：处理最终内容**
            elif response_message.content:  # 如果模型直接返回内容（通常是最终答案）
                print(f"[模型生成结果] {response_message.content}")

                # --- START: 添加 JSON 提取和解析逻辑 ---
                json_string_match = re.search(r"```json\s*(\{.*\})\s*```", response_message.content, re.DOTALL)

                if json_string_match:
                    extracted_json_content = json_string_match.group(1)
                    try:
                        final_response = json.loads(extracted_json_content)
                        print("Agent: 任务完成，成功解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError as e:
                        print(f"Agent: 提取到JSON块但解析失败: {e}")
                        print(f"尝试解析的字符串:\n{extracted_json_content}")
                        messages.append(response_message)  # 解析失败，继续对话
                else:
                    # 如果没有匹配到 ```json 块，尝试直接解析整个 content
                    try:
                        final_response = json.loads(response_message.content)
                        print("Agent: 任务完成，直接解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError:
                        print("Agent: 生成了非JSON格式内容或非Markdown JSON块，可能还在思考或出错。")
                        messages.append(response_message)  # 非JSON格式，继续对话
                # --- END: 添加 JSON 提取和解析逻辑 ---
            else:
                print("Agent: 未知响应，可能需要更多交互。")
                break

        except Exception as e:
            print(f"调用 DeepSeek API 时发生错误: {e}")
            break

    print("\n⚠️ Agent 达到最大迭代次数或未能生成最终文案。请检查Prompt或增加迭代次数。")
    return "未能成功生成文案。"

In [28]:
# 测试案例 1: 玫瑰纯露柔肤水
product_name_1 = "玫瑰纯露柔肤水"
tone_style_1 = "活泼甜美"
result_1 = generate_rednote(product_name_1, tone_style_1)

print("\n--- 生成的文案 1 ---")
print(result_1)


🚀 启动小红书文案生成助手，产品：玫瑰纯露柔肤水，风格：活泼甜美

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '玫瑰纯露柔肤水'}
[Tool Call] 查询Milvus产品数据库：玫瑰纯露柔肤水
Observation: 工具返回结果：**银杏提取焕亮眼霜**  
含有银杏叶提取物与咖啡因，有效减轻眼部浮肿，淡化黑眼圈，提升眼部肌肤的亮度与紧致度。  
规格：20g。

-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '补水保湿、舒缓肌肤、玫瑰纯露'}
[Tool Call] 模拟生成表情符号，上下文：补水保湿、舒缓肌肤、玫瑰纯露
Observation: 工具返回结果：['💦', '💧', '🌊', '✨']
-- Iteration 3 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '玫瑰纯露、柔肤水、甜美'}
[Tool Call] 模拟生成表情符号，上下文：玫瑰纯露、柔肤水、甜美
Observation: 工具返回结果：['💧']
-- Iteration 4 --
[模型生成结果] ```json
{
  "title": "💖玫瑰纯露柔肤水｜补水保湿+舒缓肌肤的神仙水！💦",
  "body": "姐妹们！我发现了一瓶神仙水！✨\n\n就是这个「玫瑰纯露柔肤水」！简直是我的肌肤救星！💧\n\n🌟 为什么我这么爱它？\n1️⃣ **超强补水**：玫瑰纯露的天然保湿力，用完皮肤水嫩嫩的，再也不怕干燥啦！\n2️⃣ **舒缓修复**：敏感肌的福音！泛红、刺痛统统消失～\n3️⃣ **提亮肤色**：坚持用了一周，肤色真的透亮了好多！\n\n💕 使用方法超简单：\n- 洁面后直接拍在脸上\n- 湿敷效果更绝！\n\n姐妹们，这瓶真的是我今年最爱的护肤品之一！快冲！🔥",
  "hashtags": ["#玫瑰纯露", "#柔肤水推荐", "#补水保湿", "#敏感肌护肤", "#平价好物"

In [30]:
# 测试案例 2: 绿茶清爽控油洁面乳
product_name_2 = "绿茶清爽控油洁面乳"
tone_style_2 = "知性温柔"
result_2 = generate_rednote(product_name_2, tone_style_2)

print("\n--- 生成的文案 2 ---")
print(result_2)


🚀 启动小红书文案生成助手，产品：绿茶清爽控油洁面乳，风格：知性温柔

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '绿茶清爽控油洁面乳'}
[Tool Call] 查询Milvus产品数据库：绿茶清爽控油洁面乳
Observation: 工具返回结果：**银杏提取焕亮眼霜**  
含有银杏叶提取物与咖啡因，有效减轻眼部浮肿，淡化黑眼圈，提升眼部肌肤的亮度与紧致度。  
规格：20g。

-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'search_web'，参数：{'query': '绿茶清爽控油洁面乳 小红书爆款文案'}
[Tool Call] 模拟搜索网页：绿茶清爽控油洁面乳 小红书爆款文案
Observation: 工具返回结果：未找到关于 '绿茶清爽控油洁面乳 小红书爆款文案' 的特定信息，但市场反馈通常关注产品成分、功效和用户体验。
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '绿茶清爽控油洁面乳 控油 清爽 温和'}
[Tool Call] 模拟生成表情符号，上下文：绿茶清爽控油洁面乳 控油 清爽 温和
Observation: 工具返回结果：['🌿', '🍃', '✨', '💧']
-- Iteration 3 --
[模型生成结果] ```json
{
  "title": "🌿绿茶控油洁面乳｜油皮救星！清爽一整天不泛油光✨",
  "body": "姐妹们！最近发现了一款超适合油皮的洁面乳——绿茶清爽控油洁面乳！🍃\n\n作为一个常年被油光困扰的混油皮，这款洁面乳简直是我的救星！温和不刺激，洗完脸超级清爽，一点都不会紧绷，而且控油效果真的绝了！早上洗完脸，到下午都不会泛油光，妆容也更服帖了～\n\n成分里添加了绿茶提取物，不仅能深层清洁，还能抗氧化，简直是一举两得！💧\n\n使用方法也超简单：\n1. 取适量洁面乳，加水揉搓出泡沫\n2. 轻轻按摩面部，重点清洁T区\n3. 温水洗净，后续护肤更吸收哦！\n\n真心推荐给所有油皮和混油皮的姐妹，夏天必备！✨",
  "hash

向量数据库效果不是很好，没搜到最相关的商品信息